# RASER MRI Sim 5
> **Code Author**: Alon Greenbaum \
> **Last Edit**: 18 July 2025 \
> **Last Edited by**: Reagan McNeill Womack \

Code based on dynamics and theory outlined in [DOI: 10.1126/sciadv.abp8483](https://doi.org/10.1126/sciadv.abp8483)

*Google Gemini generative AI was used in the development of this notebook.*

## Requirements

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.ndimage import rotate
from pathlib import Path
import os
from datetime import datetime
import random
from skimage.transform import radon, iradon
from skimage.io import imsave

ModuleNotFoundError: No module named 'matplotlib'

## Results Directory
If directory does not exist, a new directory is made at location defined with `path`.

In [2]:
def makedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

## Sample Inversion Map
Create a sample 2D inversion map with a square shape for demonstration.

### Parameters
- `shape` (tuple of `int`, default: `(10,10)`: defines size of map, includes 2 variables (1 for each direction); represents the initial distribution of population inversion in a 2D space

### Returns
- A 2D NumPy array with dimensions specified by `shape`, a simplified population inversion map

In [3]:
def createSampleInversionMap(shape=(10,10)):
  squareSize = (shape[0] // 2, shape[1] // 2) # set size of square using parameter
  inversionMap = np.full(squareSize, 1)
  inversionMap = np.pad(inversionMap, ((shape[0] // 4, shape[0] // 4), (shape[1] // 4, shape[1] // 4)), mode="constant", constant_values=0)
  return inversionMap

## Simulate RASER Dynamics
Core simulation function \
It models the time evolution of multiple interacting RASER (Radio-frequency Amplification by Stimulated Emission of Radiation) modes based on a set of coupled non-linear ordinary differential equations (ODEs), as described in the supplementary material (equations S5-S9). It takes an initial 1D population inversion profile and simulates how the population inversion, mode amplitudes, and phases change over time, ultimately calculating the total measurable output signal.

### Parameters
- `initialPopulationInversion` (numpy.ndarray): a 1D NumPy array representing teh initial population inversion $d_{\mu}(0)$ for each RASER mode/slice; this array is typically a 1D projection obtained from the 2D image
- `T1` (float, default: 5.0): the longitudinal relaxation time $T_1$ in seconds; this parameter governs how quickly the population inversion recovers to its equilibrium state (or decays if no pumping is present)
- `T2` (float, default: 0.7): the effective transverse relation time $T_2^*$ in seconds; this parameter describes the decay of the transverse magnetization (and thus the amplitude for each mode)
- `couplingBeta` (float, default: 1.0): the coupling parameter $\beta$ that dictates the strength of the interaction between the RASER modes and the overall gain; it's derived from physical constant and resonator properties
- `centerFreqHz` (float, default: 50.0): the center frequency $\nu_0$ of the RASER resonator in Hertz; this is the central frequency around which the individual RASER modes are distributed
- `gainBandwidthHz` (float, default: 10.0): the total bandwidth $\Delta$ of the imaging domain in Hertz, which corresponds to the frequency range covered by the magnetic field gradient
- `modeSpacingHz` (float, default: 0.2): the frequency separation $\delta \nu$ between adjacent RASER modes in Hertz
- `simDuration` (float, default: 2.0): the total duration of the simulation in seconds
- `pointsPerSec` (int, default: 2000): the number of data points to generate per second of simulation time for the output signal; this affects the temporal resolution of the `outputSignal`

### Returns: `dict`
A dictionary containing the results of the simulation.
- `time` (numpy.ndarray): a 1D array of time points (in seconds) at which the output signal was sampled
- `Nmodes` (int): the number of RASER modes/slices ($N$) simulated
- `initialInversion` (numpy.ndarray): the 1D array of initial population inversion values for each mode, as provided to the function
- `finalInversion` (numpy.ndarray): a 1D array of the population inversion values for each mode at the end of the simulation
- `finalAmplitude` (numpy.ndarray): a 1D array of the amplitude values ($A_\mu$) for each mode at the end of the simulation
- `outputSignal` (numpy.ndarray): a 1D array representing the total simulated RASER signal ($Sig(t)$) (see Eq. S9) over the `simDuration`

In [ ]:
def simulateRaserDynamics(
        initialPopulationInversion,
        T1=5.0,
        T2=0.7,
        couplingBeta=1.0,
        centerFreqHz=50.0,
        gainBandwidthHz=10.0,
        modeSpacingHz=0.2,
        simDuration=2.0,
        pointsPerSec=2000,
):
    N = len(initialPopulationInversion)

    # prevent running the simulation on empty projections
    if N == 0 or np.all(initialPopulationInversion == 0):
        print("Initial Population Inversion is an empty projection - simulation cancelled")
        return {
            'time': np.linspace(0, simDuration, int(simDuration * pointsPerSec)),
            'Nmodes': N,
            'initialInversion': initialPopulationInversion,
            'finalInversion': np.zeros(N),
            'finalAmplitude': np.zeros(N),
            'outputSignal': np.zeros(int(simDuration * pointsPerSec)),
        }

    print(f"Running multimode RASER simulation with {N} modes...")
    epsilon = 1e-12 # used to prevent division by zero
    muIndices = np.arrange(1, N + 1)
    naturalFrequenciesHz = centerFreqHz - 0.5 * (gainBandwidthHz - modeSpacingHz * (2 * muIndices - 1))
    omegaNaturalRad = 2 * np.pi * naturalFrequenciesHz # first term in Eq. S7

    d0 = initialPopulationInversion # initial population inversion for each mode
    A0 = np.random.uniform(0,1e-4,N) # initial amplitudes for each mode, set to tiny random values
    phi0 = np.random.uniform(0,2 * np.pi,N) # initial phases for each mode, set to random values between 0 and 2pi
    y0 = np.concatenate([d0,A0,phi0]) # create 1D array from above 3 arrays in format ODE solver expects

    def raserOdeSystem(t, y):
        d = y[0:N]
        A = y[N:2 * N]
        phi = y[2 * N:3 * N]

        X = np.sum(A * np.cos(phi))
        Y = np.sum(A * np.sin(phi))
        cosPhi = np.cos(phi)
        sinPhi = np.cos(phi)
        sumTerm1 = (X * cosPhi) + (Y * sinPhi)
        sumTerm2 = (Y * cosPhi) - (X * sinPhi)


## Test Functions
Comment/uncomment out functions to test they are working.

In [4]:
# Sample Inversion map
createSampleInversionMap()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])